## Problem 1

# a)

We want to obtain log-likelihood function for a random sample of n observations. Our likelihood function looks like this:

\begin{equation*}
L(\alpha, \beta | x ) = \Pi_{i=1}^n f(x_i | \alpha , \beta )
\end{equation*}

Now we logarithmize both sides of likelihood function and perform several computations in order to simplify result:

\begin{equation*}
\ln L(\alpha, \beta | x ) = \sum_{i=1}^n \ln f(x_i | \alpha , \beta ) = 
\sum_{i=1}^n \ln (\alpha \beta x_{i}^{\beta-1}e^{-\alpha x_i^\beta}) = 
\sum_{i=1}^n \Big( \ln(\alpha \beta) + (\beta-1) \ln(x_i)-\alpha x_i^\beta \Big) = 
n \ln (\alpha \beta ) + (\beta - 1) \sum_{i=1}^n \Big(\ln(x_i)\Big) - \alpha \sum_{i=1}^n x_i^\beta
\end{equation*}


Our log-likelihood function for a random sample of n observations:
\begin{equation*}
\ln L(\alpha, \beta | x ) = n \ln (\alpha \beta ) + (\beta - 1) \sum_{i=1}^n (\ln(x_i)) - \alpha \sum_{i=1}^n x_i^\beta
\end{equation*}

# b)

Necessary condition equations for $\alpha$ and $\beta$ can be easily derived by taking first derivatives of the log-likelihood function with respect to $\alpha$ and $\beta$ respectively and setting them to be equal to zero:\
(This makes sense, as we are trying to maximize the log-likelihood function) 


$$
\left[\begin{array}{cc} 
\frac{\partial \ln L}{\partial \alpha} \\
\frac{\partial \ln L}{\partial \beta}
\end{array}\right] =
\left[\begin{array}{cc} 
\frac{n}{\alpha} - \sum_{i=1}^n x_i^\beta \\ 
\frac{n}{\beta} + \sum_{i=1}^n \Big( \ln ( x_i ) \Big) - \alpha \sum_{i=1}^n x_i^\beta \ln(x_i)
\end{array}\right] =
\left[\begin{array}{cc} 
0 \\ 
0
\end{array}\right]
$$


From this we can easily derive that 

$$\alpha = \frac{n}{\sum_{i=1}^n x_i^\beta}$$

# c) 

Hessian matrix of the log-likelihood with respect to $\alpha$ and $\beta$:

$$
H = 
\left[\begin{array}{cc} 
\frac{\partial^2 \ln L}{\partial \alpha^2} & \frac{\partial^2 \ln L}{\partial \alpha \partial \beta}\\
\frac{\partial^2 \ln L}{\partial \alpha \partial \beta} & \frac{\partial^2 \ln L}{\partial \beta^2}
\end{array}\right] =
\left[\begin{array}{cc} 
\frac{-n}{\alpha^2} & - \sum_{i=1}^n x_i^\beta \ln(x_i)\\ 
-\sum_{i=1}^n x_i^\beta \ln(x_i) & \frac{-n}{\beta^2}-\alpha \sum_{i=1}^n x_i^\beta \ln^2(x_i)
\end{array}\right]
$$



We will load our dataset for Problem Set 1:

In [15]:
setwd("C:/Users/Honza/Desktop/ecoxhw1/ecox1")
data1 <- get(load("problem_1_dataset.Rdata"))
head(data1)
summary(data1)

[1] 7.707593 4.074502 9.548650 9.200836 4.282653 9.198474

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
  0.00902   4.44232  10.56822  15.19931  21.05152 154.02744 

# d)

We want to obtain maximum likelihood estimates of $\alpha$ and $\beta$, and estimate the asymptotic covariance matrix for the estimates. We begin by computing estimates of $\alpha$ and $\beta$:




In [43]:
library('maxLik') #load necessary packages
library('lmtest')

n <- length(data1) #number of observations

logLikFun <- function(param){
    if (param[1] <= 0 | param[2] <= 0) return(NA) #alpha and beta must beqreater than zero
    return(n* log(param[1]*param[2]) + (param[2]-1)*sum(log(data1)) - param[1]*sum(data1^param[2])) #log-likelihood function
}

mle <- maxLik(logLikFun, start = c(alpha = 2, beta = 2))
summary(mle)
#print(paste("alpha = " coef))

round(coef(mle)[1],5)
round(coef(mle)[2],5)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 15 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -18605.46 
2  free parameters
Estimates:
      Estimate Std. error t value Pr(> t)    
alpha 0.062974   0.002381   26.44  <2e-16 ***
beta  1.013935   0.011149   90.95  <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
--------------------------------------------

alpha 
0.06297

beta 
1.01394

Now that we have obtained estimates of $\alpha$ and $\beta$ we can compute covariance matrix for our estimates: